In [1]:
import numpy as np
import pandas as pd
import torch 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("../Datasets/diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
x = df.drop("Outcome", axis = 1)
y = df["Outcome"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=50)

In [4]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_test

array([[-0.85809692, -0.06603621, -1.2605803 , ...,  0.44026961,
        -0.57372814, -0.7168498 ],
       [ 1.22557572,  2.322686  ,  0.35180703, ...,  0.68759398,
         0.40828869,  1.99748145],
       [-0.85809692, -0.09705858, -0.55516084, ...,  0.1682128 ,
        -0.63113836, -0.8864955 ],
       ...,
       [-0.5604294 , -1.12079667, -0.20245112, ...,  0.94728457,
         1.39030553, -0.5472041 ],
       [ 1.82091076,  0.36827692, -0.0512898 , ..., -0.61085897,
        -0.6794838 ,  0.21620157],
       [ 2.71391332, -0.53137171,  0.15025861, ..., -0.09147779,
        -0.01473394,  0.38584727]], shape=(154, 8))

In [5]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [6]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


Defining the Model

In [7]:
import torch.nn as nn

class SimpleNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):

        z = self.linear(features)
        out = self.sigmoid(z)

        return out
    

In [8]:
learning_rate = 0.1
epochs = 100

In [9]:
loss_function = nn.BCELoss()

In [10]:
y_train_tensor.shape

torch.Size([614])

Training the Model

In [11]:
model = SimpleNN(x_train.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

for epoch in range(0, epochs):

    #Forward Propagation
    y_predict = model.forward(x_train_tensor)

    loss = loss_function(y_predict, y_train_tensor.reshape(-1, 1))

    optimizer.zero_grad()

    #Backward Propagation
    loss.backward()

    optimizer.step()

    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")


    

Epoch: 1, Loss: 0.8745754361152649
Epoch: 2, Loss: 0.8494490385055542
Epoch: 3, Loss: 0.8260868787765503
Epoch: 4, Loss: 0.8043991327285767
Epoch: 5, Loss: 0.7842897176742554
Epoch: 6, Loss: 0.7656591534614563
Epoch: 7, Loss: 0.7484065294265747
Epoch: 8, Loss: 0.7324318289756775
Epoch: 9, Loss: 0.7176375389099121
Epoch: 10, Loss: 0.7039299607276917
Epoch: 11, Loss: 0.6912205815315247
Epoch: 12, Loss: 0.6794260740280151
Epoch: 13, Loss: 0.6684688925743103
Epoch: 14, Loss: 0.6582779884338379
Epoch: 15, Loss: 0.6487876772880554
Epoch: 16, Loss: 0.6399381756782532
Epoch: 17, Loss: 0.6316750645637512
Epoch: 18, Loss: 0.6239488124847412
Epoch: 19, Loss: 0.616714358329773
Epoch: 20, Loss: 0.6099314093589783
Epoch: 21, Loss: 0.6035630702972412
Epoch: 22, Loss: 0.5975759029388428
Epoch: 23, Loss: 0.5919397473335266
Epoch: 24, Loss: 0.5866273641586304
Epoch: 25, Loss: 0.5816139578819275
Epoch: 26, Loss: 0.5768768787384033
Epoch: 27, Loss: 0.5723959803581238
Epoch: 28, Loss: 0.5681523680686951
Ep

Model Evaluation

In [12]:
with torch.no_grad():
    out = model.forward(x_test_tensor)
    _, y_predict = torch.max(out, 1)
    accuracy = accuracy_score(y_test_tensor.reshape(-1, 1), y_predict)
    print(accuracy)

0.6558441558441559
